In [1]:
import numpy as np
import pickle
from bitarray import bitarray
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM, Embedding, Bidirectional
from keras.layers import Activation
from keras.optimizers import *
from keras.activations import *
from keras.layers.advanced_activations import *
from pandas import Series
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, TensorBoard
import os
import utils
import matplotlib.pyplot as plt
from keras import regularizers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
import keras.utils.np_utils as np_utils
import utils

Using TensorFlow backend.


In [136]:
data = pickle.load(open("dataset/folk_music_803.notes", "rb"))
data = np.array(data)

In [137]:
len(data)

103664

In [138]:
len(set(data))

79

In [139]:
data = data.reshape(-1, 1)

In [140]:
#data = data[0: 100]

In [141]:
enc = LabelEncoder()
enc = enc.fit_transform(data)

In [142]:
onehot = OneHotEncoder(sparse=False)
enc = enc.reshape(len(enc), 1)
transformed = onehot.fit_transform(enc)

/home/lukasz/.local/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [143]:
sequence_length = 50
network_input = []
network_output = []

In [144]:
for i in range(0, len(transformed) - sequence_length, 1):
        sequence_in = transformed[i:i + sequence_length]
        sequence_out = transformed[i + sequence_length]
        network_input.append(sequence_in)
        network_output.append(sequence_out)

In [145]:
network_input = np.array(network_input)
network_output = np.array(network_output)

In [150]:
network_input.shape

(103614, 50, 79)

In [151]:
network_input[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [152]:
network_input.shape

(103614, 50, 79)

In [153]:
network_output[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [154]:
model = Sequential()
model.add((LSTM(128, input_shape=(network_input.shape[1], network_input.shape[2]), return_sequences=True,
                # kernel_regularizer=regularizers.l2(0.01),
                # activity_regularizer=regularizers.l1(0.01)
)))
model.add(LSTM(64))
# model.add(Dropout(0.5))
model.add(Dense(79))
# model.add(Activation('linear'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [155]:
results = utils.create_results_dir()
callbacks_list = utils.model_callbacks(results)
history = model.fit(network_input, network_output,
                    epochs=100,
                    batch_size=64,
                    verbose=1,
                    shuffle=True,
                    callbacks=callbacks_list
                    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
103614/103614 [==============================] - 160s 2ms/step - loss: 8.8001 - acc: 0.1058
Epoch 2/100
  5632/103614 [>.............................] - ETA: 2:30 - loss: 8.6486 - acc: 0.1241

KeyboardInterrupt: 